<a href="https://colab.research.google.com/github/PravinC0d3R28/Train_LLM_on_Documentation/blob/main/NanoGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Training LLM on Javascript Documentation

Cloning nanoGPT repo

In [ ]:
!pip install torch --upgrade

In [ ]:
# gpt2 uses transformers
%pip install tiktoken transformers wandb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00


In [ ]:
#!wandb login (optional)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
Aborted!


View and prepare the data

In [ ]:
!head ./nanoGPT/data/javascript/extracted_data.txt

Web technology reference for developers
Structure of content on the web
Code used to describe document style
General-purpose scripting language
Protocol for transmitting web resources
Interfaces for building web applications
Developing extensions for web browsers
Web technology reference for developers
Learn web development
Learn web development


In [ ]:
!cd ./nanoGPT/data/extracted_data/ && python prepare.py

length of dataset in characters: 6,544
all the unique characters: 
 "'(),-.012345689:=?ABCDEFGHIJLMNOPRSTUVWYabcdefghijklmnopqrstuvwxyz©–—’
vocab size: 73
train has 5,889 tokens
val has 655 tokens


Setup the configuration for training - you have a config file and you can pass arguments at training time

In [ ]:
!cat ./nanoGPT/config/train_javascript_char.py

# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-javascript-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'vendata'
wandb_run_name = 'nano-gpt'

dataset = 'extracted_data'
gradient_accumulation_steps = 1
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup_iters = 100 # not super necessary po

Train the model

In [ ]:
!cd nanoGPT && python train.py config/train_javascript_char.py

Overriding config with config/train_javascript_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-javascript-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'vendata'
wandb_run_name = 'nano-gpt'

dataset = 'extracted_data'
gradient_accumulation_steps = 1
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per it

Review the samples

In [ ]:
!cd ./nanoGPT && python sample.py --out_dir=out-javascript-char --ckpoint='ckpt_iter_250.pt' --start="what is javascript?"

Overriding: out_dir = out-javascript-char
Overriding: ckpoint = ckpt_iter_250.pt
Overriding: start = what is javascript?
number of parameters: 10.65M
Loading meta from data/extracted_data/meta.pkl...
what is javascript?", and "what does is is JavaScript?", "what does it loke?", and "what it doescumeng ke?", along with discussing key JavaScript features such as variables, strings, numbers, and arays.
Continues our coverage of JavaScript's key fundamental features, turning our attten aton commmonly-encountered types of code ble blocks s such as condional statatements, loooops, functions, and events.
The object-oriented nature of JavaScript is important t u to wan go f function lebl pager opanertiesonces (ve for.
---------------
what is javascript?", "what does it is JavaScript?", "what does it documpleng es oke?", along withat discussing key JavaScript features such as variables, strings, numbers, and arays.
Continues our coverage of JavaScript's key fundamental features, turning our att

In [ ]:
!cd ./nanoGPT && python sample.py --out_dir=out-javascript-char --ckpoint='ckpt_iter_1000.pt' --start="what is javascript"

Overriding: out_dir = out-javascript-char
Overriding: ckpoint = ckpt_iter_1000.pt
Overriding: start = what is javascript
number of parameters: 10.65M
Loading meta from data/extracted_data/meta.pkl...
what is javascript ons and related technologies, see JavaScript technologies overview.
Looking to become a front-end web developer?

    We have put together a course that includes all the essential information you need to
    work towards your goal.
  
Get started
Learn how to program in JavaScript with guides and tutorials.
Head over to our Learning Area JavaScript topic if you want to learn JavaScript but have no previous experience with JavaScript or programming. The complete modules available there are as fo
---------------
what is javascript (JS) is a lightweight interpreted (or just-in-time compiled) programming language with first-class functions. While it is most well-known as the scripting language for Web pages, many non-browser environments also use it, such as Node.js, Apache 

In [ ]:
!cd ./nanoGPT_simpsons && python sample.py --out_dir=out-javascript-char --ckpoint='ckpt_iter_5000.pt' --start="standards for JavaScript"

Overriding: out_dir = out-javascript-char
Overriding: ckpoint = ckpt_iter_5000.pt
Overriding: start = standards for JavaScript
number of parameters: 10.65M
Loading meta from data/extracted_data/meta.pkl...
standards for JavaScript are the ECMAScript Language Specification (ECMA-262) and the ECMAScript Internationalization API specification (ECMA-402). As soon as one browser implements a feature, we try to document it. This means that cases where some proposals for new ECMAScript features have already been implemented in browsers, documentation and examples in MDN articles may use some of those new features. Most of the time, this happens between the stages 3 and 4, and is usually before the spec is officially published.

---------------
standards for JavaScript are the ECMAScript Language Specification (ECMA-262) and the ECMAScript Internationalization API specification (ECMA-402). As soon as one browser implements a feature, we try to document it. This means that cases where some prop